In [ ]:
%pip install python-dotenv
%pip install -U scikit-learn

In [ ]:

from typing import Dict


from models.econometrics import Cost, Currency

import math
from typing import Callable
from models.components import Component
from models.econometrics import Cost
from models.geometry import Orientation
from models.photovoltaic import Photovoltaic

In [ ]:
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Currencies values

In [ ]:
import requests
import json
#currency values

# national units https://mindicador.cl/api
query_factor:requests.Response = requests.get('https://mindicador.cl/api',timeout=1000)
ratios_cl = json.loads(query_factor.text)
usd_clp:float = ratios_cl['dolar']['valor']
Cost.set_exchange(Currency.CLP,usd_clp)
Cost.set_exchange(Currency.UF,usd_clp/ratios_cl['uf']['valor']) # 1 dolar in UF
Cost.set_exchange(Currency.UTM,usd_clp/ratios_cl['utm']['valor']) # 1 dolar in Utm

# exchange rates https://app.freecurrencyapi.com/dashboard
query_exchange:requests.Response = requests.get(config["CURRENCY_API_KEY"],timeout=1000)

currency_ratios = json.loads(query_exchange.text)
Cost.set_exchange(Currency.EUR,currency_ratios['data']['EUR'])
Cost.set_exchange(Currency.GBP,currency_ratios['data']['GBP'])
Cost.set_exchange(Currency.BRL,currency_ratios['data']['BRL'])


## Address and general info🌎

In [ ]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

project:Project = Project(
    title='Sistema Fotovoltaico OnGrid-Mixto',
    building = Building(
        geolocation=GeoPosition(latitude=-33.861702,longitude=-71.657839),
        name='Posta de Salud Rural Bucalemu',
        address='Ruta G-80-I km 30 sn, Localidad Bucalemu,\nSanto Domingo',
        city='Santo Domingo'),
    technology= [Tech.PHOTOVOLTAIC]
    )

# psr_sanenrique:Project = Project(
#     title='Sistema Fotovoltaico OnGrid-Mixto PSR San Enrique, Santo Domingo',
#     building = Building(
#         geolocation=GeoPosition(latitude=-33.9081421,longitude=-71.7159192),
#         name='Posta de Salud Rural de San Enrique',
#         address='Ruta G-866 km 0 sn, Localidad San Enrique, Santo Domingo',
#         city='Santo Domingo'),
#     technology= [Tech.PHOTOVOLTAIC]
#     )



In [ ]:
project.weather.get_data().head(24)

# Integration 

## Energy Consumptions🔌

In [ ]:
from models.consumption import ElectricityBill, Energetic
project.building.add_consumptions(description="main",energetic=Energetic.ELI,consumption=[
    ElectricityBill(450,"30-12-2022",cost=71400),
    ElectricityBill(497,"30-01-2023",cost=80800),
    ElectricityBill(450,"28-02-2023",cost=71400),
    ElectricityBill(475,"30-03-2023",cost=77400),
    ElectricityBill(475,"30-04-2023",cost=77400),
    ElectricityBill(520,"30-05-2023",cost=93400),
    ElectricityBill(654,"30-06-2023",cost=122100),
    ElectricityBill(820,"30-07-2023",cost=140800),
    ElectricityBill(549,"31-10-2023",cost=100000),
])

In [ ]:
project.building.consumptions['main'].forecast()

## Machine Components 🎡

In [ ]:
#cspell: disable

#PV COST MODEL, inscludes price clp per watt, including cost of planel, inverter, instalation and infraestructure.
pv_log_model_on_grid:Callable[[float],float] = lambda size_w: math.floor(-152.6*math.log(size_w)+2605.9) # clp/w
pv_log_model_off_grid:Callable[[float],float] = lambda size_w: math.floor(-414.7*math.log(size_w)+6143.9) # clp/w
#PV linear COST MODEL, inscludes price clp per watt,just panel.
pv_linear:Callable[[float],float] = lambda size_w: 245990/655

#clean previous load
project.components = {}
from IPython.display import clear_output
clear_output(wait=True)

# assembly projects components
project.add_component(
    'generación',
    Photovoltaic(project.weather,'Módulo Primario A',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                power=655,
                quantity=5,
                cost=None,
                cost_model=pv_linear,
                orientation=Orientation(15,270-16.46)),
    Photovoltaic(project.weather,'Módulo Secundario B',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                power=655,
                quantity=5,
                cost=None,
                cost_model=pv_linear,
                orientation=Orientation(15,16.46))
    )
project.add_component(
    'auxiliar',
    Component('inversor de 7kW',
                model='Voltronic 7kW 48V',
                reference='https://solarbex.com/comprar/inversor-hibrido-7kw-48v-voltronic/',
                specification='Híbrido 7kW Monofásico',
                cost_per_unit=Cost(1255,Currency.EUR)),
    Component('Controlador de Carga',
                model='Enertik 80A 12/24V',
                specification='1000W to 2000 24V output',
                cost_per_unit=Cost(101600,Currency.CLP)),
    Component('medidor bidireccional',
                model='Elster AS1440',
                specification='Trifasico',
                cost_per_unit=Cost(306990,Currency.CLP)),
    Component('almacenamiento',
                model='MP NP12-200',
                specification='Betería 12V 200Ah',
                reference='https://www.tiendatecnored.cl/bateria-gel-ciclo-profundo-12v-200ah.html',
                cost_per_unit=Cost(204889,Currency.CLP),
                quantity=10,
                )
    )

project.add_component(
    'obras',
    Component(description='instalación eléctrica',
            specification='H/P eléctrico',
            cost_per_unit=Cost(1,currency=Currency.UF),
            quantity=100,
            ),
    Component(description='estructura de soporte',
            specification='elementos fijación',
            cost_per_unit=Cost((4*800+11400/4+4*700+4000+500)),
            quantity=24
            )
)

In [ ]:
# psr_bucalemu.components['generación'][1].nominal_power()
#pv_log_model(1500) #clp/w

bucket_list = project.bucket_list(Currency.CLP)
    
#unitary price comparison
print(
    'estimado',bucket_list['cost'],
    'on grid',pv_log_model_on_grid(655*10)*655*10,
    'off grid',pv_log_model_off_grid(655*10)*655*10,
    'linear',pv_linear(655*10)*655*10)



In [ ]:
# psr_bucalemu.get_energy_generation('generación')
project.get_energy('generación').head(10)

In [2]:
from models.emission import emission
emission.annual_avg()
emission.annual_projection(2024)

KeyError: 'year'

# Write Document 📒


In [ ]:
from docxtpl import * 


In [ ]:
#assembly proxy object 
ctx:dict[str,any] = {
    #about this project
    "project_name": project.title,
    "project_type" : project.technology[0],
    "project_size":"*test",
    "size_unit":"kW",
    "total_cost": project.bucket_list(Currency.CLP)["cost"],
    #site
    "building_name" : project.building.name,
    "city": project.building.city
}

In [ ]:
from docxtpl import DocxTemplate
doc = DocxTemplate("templates/memory_template.docx")
doc.render(ctx)
doc.save(f"build/reporte_{project.building.city}_{project.building.name}.docx")

project.get_context('generación')